In [13]:

"""
Created on Wed Sep 11 16:13:02 2019

The code presented here is a simplified piece of code used for traiging of screening breast MRI in women with extremly dense breast
as published in Radiology 2021 "Deep Learning for Automated Triaging of 4581 Breast MRI Examinations from the DENSE Trial"

Citation: {Verburg, E., van Gils, C. H., van der Velden, B. H., Bakker, M. F., Pijnappel, R. M., Veldhuis, W. B., & Gilhuijs, K. G. (2022). Deep learning for automated triaging of 4581 breast MRI examinations from the DENSE trial. Radiology, 302(1), 29-36.}

#INPUT
Multiple input files are required to run the code. 
-The model weights
-Input images

The  model weights resulted form the train data which was used in for publication is enclosed. Because 8 fold internal external valiation
was completed 8 weightfiles are present. In addition to that also a weight file is added were all data is used to train the model.

Enclosed are also three example images which can be traiged by the model. The images are Maximal intensity projection (MIP) of one breast in Sagital
transversal and coronal direction. The MIP images are prepared as described in the publication. In the outcomes in the publication was the average outcome over the three MIP directions.

#OUTPUT
Output of the model are probabilities for lesion presence and SHAP images showing the areas on which the prediction in based.



#############################################
Copyright (c) 2023 UMC Utrecht

 Permission is hereby granted, free of charge, to any person
 obtaining a copy of this software and associated documentation
 files (the "Software"), to deal in the Software without
 restriction, including without limitation the rights to use,
 copy, modify, merge, publish, distribute, sublicense, and/or sell
 copies of the Software, and to permit persons to whom the
 Software is furnished to do so, subject to the following
 conditions:

 The above copyright notice and this permission notice shall be
 included in all copies or substantial portions of the Software.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
 EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
 OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
 NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY,
 WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
 FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR
 OTHER DEALINGS IN THE SOFTWARE.
################################################

@author: Erik Verburg
"""

#inputs:
# model weights can be downloaded from: https://github.com/Lab-Translational-Cancer-Imaging/AI_TriagingDENSE
model_weights=#"Weightfiles\weights_1_modelweighs.hdf5"


#Input image files, MIP images in three directions:
filename_cor=#"ExampleImages\left_Uptake0_MIP_cor.nii.gz"
filename_tra=#"ExampleImages\left_Uptake0_MIP_tra.nii.gz"
filename_sag=#"ExampleImages\left_Uptake0_MIP_sag.nii.gz"

#Import settings and functions
import numpy as np
import keras
import glob
import random
import nibabel as nib
from skimage.transform import resize
import os
import csv
import time
import shutil
import SimpleITK as sitk
import PIL
import shap
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard
from keras import backend as K
from tf_explain.callbacks.grad_cam import GradCAMCallback
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array



def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op =tf.metrics.auc(y_true, y_pred)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value
    
def specificity_at_sensitivity(sensitivity, **kwargs):
    def metric(labels, predictions):
        # any tensorflow metric
        value, update_op = tf.compat.v1.metrics.specificity_at_sensitivity(labels, predictions, sensitivity, **kwargs)

        # find all variables created for this metric
        metric_vars = [i for i in tf.local_variables() if 'specificity_at_sensitivity' in i.name.split('/')[2]]

        # Add metric variables to GLOBAL_VARIABLES collection.
        # They will be initialized for new session.
        for v in metric_vars:
            tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

        # force to update metric values
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
            return value
    return metric        

keras.metrics.auc_roc = auc_roc
keras.metrics.metric = specificity_at_sensitivity(0.9,num_thresholds=2000)

In [14]:
#Set up of the deeplearning model and load the modelweights.

#Optimal settings obtained using NNI.
Direction="25Dimesional"
View="Single"
layer_size= 128
conv_layer = 5
optimizer="Adam"
conv_size=3
lossfunction="crossentropy"
lrate=0.00008122008474925512
batchsize=2
augmentation="none"
NConvlayersinBlock=2
DropoutRate=0.16491226911610724
DenseLayer=0
DenseLayer_size=16
n_channels=1
normalisation="Yes"
dilation=0
dense_layer=0
dropout=0




model = Sequential()

model.add(Conv2D(layer_size, (conv_size, conv_size), padding = 'same',activation='relu', input_shape=(256, 256, n_channels)))
model.add(Conv2D(layer_size, (conv_size, conv_size), padding = 'same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(layer_size, (conv_size, conv_size), padding = 'same',activation='relu'))
model.add(Conv2D(layer_size, (conv_size, conv_size), padding = 'same',activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


dilationrate=1
   
for l in range(conv_layer-2):
    
    model.add(Conv2D(layer_size, (conv_size, conv_size),dilation_rate=dilationrate, padding = 'same',activation='relu'))
    model.add(Conv2D(layer_size, (conv_size,conv_size),dilation_rate=dilationrate, padding = 'same',activation='relu'))
    if NConvlayersinBlock>2:
        model.add(Conv2D(layer_size, (conv_size, conv_size), padding = 'same',activation='relu'))
    if  NConvlayersinBlock>3:   
        model.add(Conv2D(layer_size, (conv_size, conv_size), padding = 'same',activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())


model.add(Dense(2,activation='softmax'))
#model.add(Dense(2,activation='sigmoid'))



if lossfunction=='crossentropy':
    if optimizer=='Adam':
        print("start compilation")
        model.compile(loss='binary_crossentropy',
           optimizer=keras.optimizers.Adam(lr = lrate),
           metrics=['accuracy',auc_roc,specificity_at_sensitivity(0.9,num_thresholds=2000)])
    


model.summary()


              
model.load_weights(model_weights)

start compilation
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 256, 256, 128)     1280      
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 256, 256, 128)     147584    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 64, 64, 128)       147

In [15]:
#Define functions for image loading

def Niigz2NP(filename):
    reader = sitk.ImageFileReader()

    reader.SetFileName(filename)
    image = reader.Execute();
    array = sitk.GetArrayFromImage(image)
    if len(array.shape)==2:
        array = np.swapaxes(array, 0, 1)
    elif len(array.shape)==3:
        array = np.swapaxes(array, 0, 2)
    
    array=array.astype('int16') 
    
    return(array)


def SaveNP2Nii(array,outname):
    
    if len(array.shape)==2:
        array = np.swapaxes(array, 0, 1)
    elif len(array.shape)==3:
        array = np.swapaxes(array, 0, 2)
    
    im = sitk.GetImageFromArray(array, isVector=False)
    writer = sitk.ImageFileWriter()
    writer.SetFileName(outname)
    writer.Execute(im)
    

if "Yes" in normalisation:
    def norma(data):
        return((data - data.mean()) / data.std())
else:
    def norma(data):
        return(data)

def padImg(img, patchSize):
    """Zero-pads images TO a certain size

    Parameters
    ----------
    img : ndarray
        Image to be padded
    patchSize : tuple or ndarray, optional
        Patchsize of output.
        Note: behaviour with odd patchsizes (i.e. not even) undefined

    Returns
    -------
    ndarray
        The padded image
    """
    # First determine amount of dimensions unspecified
    nFullDims = img.ndim - len(patchSize)

    # if not all(pat > im for im, pat in zip(reversed(img.shape),
    #                                        reversed(patchSize))):
    #     print(img.shape, patchSize)
    #     raise ValueError('Doesnt fit?')

    # First determine dimensions of new array
    #  WHICH IS patchsize
    # new_size = img.shape[:nFullDims] + \
    #     tuple(reversed(tuple(pat for pat in reversed(patchSize))))
    # Initialize array
    padimg2 = np.zeros(img.shape[:nFullDims] + patchSize)
    
    """
    Below looks quite complicated, but there is logic behind it!
    Basically we have two kinds of dimensions: the first kind are
    the dimensions that should be copied over in full without
    padding or anything, as they are not specified in the patch size.
    The dimensions that ARE to be padded, we use the reverse-reverse
    trick to determine the starting and ending position. The ending
    position is half the patchsize (== padding size), end size is trivial.
    """
    padimg2[tuple([*tuple(slice(0, p) for p in img.shape[:nFullDims]),
                   *tuple(reversed(tuple(
                       slice(max(0, (pat - im) // 2),
                             max(0, (pat - im) // 2) + im)
                       for im, pat in zip(reversed(img.shape[nFullDims:]),
                                          reversed(patchSize))
                   )
                   ))])] = \
        img[tuple([*tuple(slice(0, p) for p in img.shape[:nFullDims]),
                   *tuple(reversed(tuple(
                       slice(0, min(im, pat))  # Get smallest dim
                       for im, pat in zip(reversed(img.shape[nFullDims:]),
                                          reversed(patchSize))
                   )
                   ))])]

    
    return padimg2




In [16]:
#apply model on images

IDS=[filename_cor[0],filename_tra[0],filename_sag[0]]

dim1=256
dim2=256
x_test=np.empty((1,dim1,dim2,1))
x_test[0,:,:,0]=norma(padImg(Niigz2NP(filename_cor[0]),(dim1,dim2)))
outcome_cor= model.predict(x_test)

print("probabilty lesion presence in coronal uptake MIP = "  + str(outcome_cor[0][1]))

dim1=256
dim2=256
x_test=np.empty((1,dim1,dim2,1))
x_test[0,:,:,0]=norma(padImg(Niigz2NP(filename_tra[0]),(dim1,dim2)))
outcome_tra= model.predict(x_test)

print("probabilty lesion presence in transversal uptake MIP = "  + str(outcome_tra[0][1]))

dim1=256
dim2=256
x_test=np.empty((1,dim1,dim2,1))
x_test[0,:,:,0]=norma(padImg(Niigz2NP(filename_sag[0]),(dim1,dim2)))
outcome_sag= model.predict(x_test)

print("probabilty lesion presence in sagital uptake MIP = "  + str(outcome_sag[0][1]))
print("\n")
print("Average probability for lesions presence over three directions = " + str((outcome_sag[0][1]+outcome_tra[0][1]+outcome_cor[0][1])/3))



probabilty lesion presence in coronal uptake MIP = 0.48781636
probabilty lesion presence in transversal uptake MIP = 0.9816363
probabilty lesion presence in sagital uptake MIP = 0.91186816


Average probability for lesions presence over three directions = 0.7937735716501871


In [ ]:
#use SHAP to display the areas were the probabilty for lesion presence is based on.

%matplotlib inline

dim1=256
dim2=256
numberofbackgrounds=3

iter=0
x_train=np.empty((numberofbackgrounds,dim1,dim2,1))

for ID in IDS[0:numberofbackgrounds]:
    x_train[iter,:,:,0]=norma(padImg(Niigz2NP(ID ),(dim1,dim2)))

    iter+=1

background = x_train

# explain predictions of the model 
e = shap.DeepExplainer(model, background)

print('succes')

iter=0
for ID in IDS:
    x_test[iter,:,:,0]=norma(padImg(Niigz2NP( ID ),(dim1,dim2)))
    

    print(ID)
    
    shap_values = e.shap_values(x_test)
    
    shap.image_plot(shap_values, x_test)    